In [4]:
import logging

logger = logging.getLogger()
logger.handlers.clear()

# format
formatter = logging.Formatter('[%(asctime)s] [%(levelname)s] %(message)s')

# add stream handler
handler = logging.StreamHandler(sys.stdout)                             
handler.setFormatter(formatter)
logger.addHandler(handler)

# enable all logging level
logger.setLevel(logging.INFO)

Definition for the board.

In [2]:
import os

os.environ["OKFP_ROOT"] = "C:/Users/sean9/Downloads/FrontPanelUSB"

from ece437.ok import OKFrontPanel

path = "C:/Users/sean9/ECE437/Final/Final.runs/impl_1/final_top.bit"
fp = OKFrontPanel(firmware_path=path)

Start control DRV8833 over PMOD port 1.

In [5]:
from ece437.presets import I2C_1_ENDPOINTS, PMOD_1_ENDPOINTS
from ece437.i2c import I2CController
from ece437.sensor import LSM303Accelerometer, LSM303Magnetometer
from ece437.pmod import PMOD, PMODType5Endpoints, PMODDirection
import time
import numpy as np

with fp:
    logger.info(f"serial={fp.serial_number}")

    with I2CController(fp, I2C_1_ENDPOINTS) as i2c1:
        acc = LSM303Accelerometer(i2c1)
        acc.configure() # defaults

        mag = LSM303Magnetometer(i2c1)
        mag.configure() # defaults

        n_measurements = 100
        n_repeats = 10

        acc_max_hist = []

        for i in range(n_repeats):
            logger.warning(f'trial {i+1}')
            acc_hist = []

            with PMOD(fp, PMOD_1_ENDPOINTS) as pmod:
                logger.info('reset')
                pmod.start(2000, PMODDirection.Forward, block=True)

                logger.info('start moving')
                pmod.start(200, PMODDirection.Reverse)
                logger.info('start recording')
                for i in range(n_measurements):
                    acc_hist.append(acc.read())

            acc_hist = np.array(acc_hist)

            acc_max = abs(acc_hist).max(axis=0)
            acc_max_hist.append(acc_max)
        
        acc_max_hist = np.array(acc_max_hist)
        print(f'avg={acc_max_hist.mean(axis=0)}')
        print(f'std={acc_max_hist.std(axis=0)}')
        

[2022-12-15 22:05:35,686] [INFO] found 1 devices on this system
[2022-12-15 22:05:35,694] [INFO] upload firmware 'C:/Users/sean9/ECE437/Final/Final.runs/impl_1/final_top.bit'
[2022-12-15 22:05:35,826] [INFO] serial=1911000P3V
[2022-12-15 22:05:35,843] [WARNING] trial 1
pmod opened
[2022-12-15 22:05:35,844] [INFO] reset
[2022-12-15 22:05:35,846] [INFO] motor start
[2022-12-15 22:05:35,846] [INFO] blocking, wait for is_motor_done
[2022-12-15 22:05:37,846] [INFO] start moving
[2022-12-15 22:05:37,846] [INFO] motor start
[2022-12-15 22:05:37,847] [INFO] start recording
[2022-12-15 22:05:38,264] [WARNING] trial 2
pmod opened
[2022-12-15 22:05:38,265] [INFO] reset
[2022-12-15 22:05:38,265] [INFO] motor start
[2022-12-15 22:05:38,267] [INFO] blocking, wait for is_motor_done
[2022-12-15 22:05:38,465] [INFO] start moving
[2022-12-15 22:05:38,466] [INFO] motor start
[2022-12-15 22:05:38,466] [INFO] start recording
[2022-12-15 22:05:38,881] [WARNING] trial 3
pmod opened
[2022-12-15 22:05:38,882] 